# Australian AA Variants

## Part 00: Extract and Transform Metadata

The following cells manipulate the GISAID source metadata file,\
extracting sequences collected in Austrailian Regions/Territories.

The location of the metadata file (`metadata.tsv.xz`) is specified in\
the cell under "Definitions > Filepaths".

Several transformed files featuring general summaries will be created\
for use in further analyses.

##### General Sequence
From the metadata, accession ids are categorized by quarter-year periods (Q1, ...)

Using the union of all observed substitutions across all sequences as features, \
amino acid substitutions are coded in a binary manner for each sequence.

Then, comparing the count of *all* peer sequences in the region-period,\
will provide the incidence of each substitution.

In [1]:
from pathlib import Path
import pandas as pd

import helpers

### Definitions

#### Filepaths

In [34]:
DATA_DIR = Path('./data')

gisaid_metadata_fpath = DATA_DIR / 'metadata_sample.tsv'
au_metadata_fpath = DATA_DIR / 'au_metadata_sample.tsv'
au_reduced_metadata_fpath = DATA_DIR / 'au_reduced_metadata_sample.tsv'

SUMMARY_DIR = DATA_DIR / 'summaries'
aa_subs_by_seq_fpath = SUMMARY_DIR / 'aa_subs_by_seq.csv'
aa_sub_counts_by_region_period_fname = SUMMARY_DIR / 'aa_sub_counts_by_regional_period.csv'
seq_counts_fpath = SUMMARY_DIR / 'seq_counts_per_regional_period.csv'

# Uncomment to create the directories
# SUMMARY_DIR.mkdir(exist_ok=True, parents=True)

#### DataFrame Column and Index Levels and Names

In [3]:
index_levels = [0,1,2,3,4]
index_names=['region', 'collection_date', 'virus_code', 'accession_id', 'time_period']
short_index_names = ['region', 'time_period']
header_levels=[0,1,2,3]
header_names = ['gene', 'position', 'aa_ref', 'aa_sub']

### Extract, Clean and Condense Metadata

In [4]:
if au_metadata_fpath.exists():  # load cached data
    au_md_df = pd.read_csv(
        au_metadata_fpath,
        sep='\t',
        **helpers.CleanedMetadataImportConfig.import_args
    ).fillna(helpers.CleanedMetadataImportConfig.nafills)
else:
    md_df = pd.read_csv(
        gisaid_metadata_fpath,
        sep='\t', 
        **helpers.RawMetadataImportConfig.import_args
    ).fillna(helpers.RawMetadataImportConfig.nafills)

    # Filter for human hosts only
    md_df = md_df[md_df.Host == 'Human']

    md_df = helpers.format_column_names(md_df)

    # Locations must include "Australia"
    au_md_df = md_df[md_df['location'].str.contains(pat='[A|a]ustralia')]
    # Get all "complete" Australian human host data 
    au_md_df = au_md_df[
        (au_md_df['is_low_coverage'] != True) &
        (au_md_df['is_complete']==True)
    ]

    au_md_df = au_md_df[[
        'accession_id',
        'virus_name',
        'location',
        'collection_date',
        'submission_date',
        'clade',
        'variant',
        'aa_substitutions',
    ]].sort_values('collection_date')

    # Cache
    au_md_df.to_csv(
        au_metadata_fpath,
        sep='\t',
        index=False
    )


In [5]:
au_md_df.sample(5)

,accession_id,virus_name,location,collection_date,submission_date,clade,variant,aa_substitutions
546,EPI_ISL_9362964,hCoV-19/Australia/NSW-ICPMR-19829/2022,Oceania / Australia / New South Wales / Sydney,2022-01-12,2022-02-01,GRA,VOC Omicron GRA (B.1.1.529+BA.*) first detecte...,"(NSP5_P132H,Spike_H69del,Spike_T95I,Spike_A67V..."
214,EPI_ISL_3187212,hCoV-19/Australia/NSW-ICPMR-1316/2021,Oceania / Australia / New South Wales / Sydney,2021-07-27,2021-08-02,GK,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,N_D63G,N_R203M,NSP12_G671S,NSP16_Q238..."
443,EPI_ISL_7772590,hCoV-19/Australia/VIC28863/2021,Oceania / Australia / Victoria,2021-12-08,2021-12-17,GK,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,NSP4_S34F,Spike_T95I,N_D63G,N_R203M,N..."
253,EPI_ISL_3834119,hCoV-19/Australia/NSW-ICPMR-5506/2021,Oceania / Australia / New South Wales / Sydney,2021-08-16,2021-09-01,GK,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,NS3_A59V,N_D63G,N_R203M,NSP2_V258I,NS..."
36,EPI_ISL_520750,hCoV-19/Australia/VIC3037/2020,Oceania / Australia / Victoria,2020-07-03,2020-08-25,GR,,"(Spike_S477N,N_R203K,N_G204R,NSP2_I120F,NSP12_..."


#### Note on Collection Date vs Submission Date

Preferably, all samples would be assigned a quarter based on\
the Collection Date alone.

However, some regions have many collection dates which\
are set to January 1st of a given year (e.g., 2020, 2021, 2022)

Since it is unlikely that all sequences were collected on Jan. 1st (especially Jan. 1st, 2020),
which happens to be the default date-time if only a year is provided on submission,
it is assumed that these are information deficient for purposes of defining a specific quarter year.

This error was discovered after reviewing a first pass of the data,
finding some apparent Jan. 1st collection dates would place variants (such as omicron) \
quarters before initial detection. 

With the extreme unlikelihood of a previously unknown variant having 10s-to-100s of samples 
all collected on January 1st without being reported for more than half a year, especially with\
the date also being the default if only a year was provided upon submission, \
it is assumed the submission date was reasonably soon after the actual collection.

Given the proportion of samples from certain regions which only provided the year,
the Submission Date was instead used as a proxy. This comes with a few limitations and caveats:

1. The time between the collection and submission occured across a quarter.
2. The time between the collection and submission was exceptionally long, i.e., across more than one quarter

However, results after the adjustment generally align with known origin dates of variants,
and so this heuristic was sufficient for purposes of this analysis.

In [7]:
if au_reduced_metadata_fpath.exists():  # load cached data
    au_reduced_df = pd.read_csv(
        au_reduced_metadata_fpath,
        sep='\t'
    )
else:
    au_reduced_df = au_md_df.copy()

    # filter all January 1st dates
    jan_first_filter = (
        (au_reduced_df.collection_date == pd.to_datetime('2020-01-01')) |
        (au_reduced_df.collection_date == pd.to_datetime('2021-01-01')) |
        (au_reduced_df.collection_date == pd.to_datetime('2022-01-01'))
    )
    au_reduced_df.loc[jan_first_filter, 'collection_date'] = au_reduced_df['submission_date']

    # Split into year, month, and day columns (day is arbitrary)
    au_reduced_df['year'] = au_reduced_df['collection_date'].dt.year
    au_reduced_df['month'] = au_reduced_df['collection_date'].dt.month
    au_reduced_df['day'] = au_reduced_df['collection_date'].dt.day

    # split location string into separate columns
    expanded_loc_cols = ['continent', 'country', 'region', 'city']
    au_reduced_df[expanded_loc_cols] = au_reduced_df['location'].str.split('/', expand=True)

    # Format to succinct "Virus Code"
    au_reduced_df['virus_code'] = helpers.get_formatted_virus_code(au_reduced_df['virus_name'])

    # Create columns for deliniating region-periods
    au_reduced_df['quarter'] = au_reduced_df.apply(helpers.get_quarter, axis=1)
    au_reduced_df['time_period'] = au_reduced_df.apply(helpers.get_period, axis=1)

    au_reduced_df['region'] = au_reduced_df['region'].str.strip()
    au_reduced_df['region'] = au_reduced_df['region'].str.replace('?', '', regex=False)

    au_reduced_df = au_reduced_df[[
        'accession_id',
        'virus_name',
        'virus_code',
        'collection_date',
        'submission_date',
        'year',
        'month',
        'day',
        'quarter',
        'time_period',
        # 'continent',  # must be Australia
        # 'country',  # must be Australia
        'region',
        'clade',
        'variant',
        'aa_substitutions',
    ]]

    au_reduced_df = au_reduced_df.sort_values('collection_date')

    # cache
    au_reduced_df.to_csv(
        au_reduced_metadata_fpath,
        sep='\t',
        index=False
    )


In [8]:
au_reduced_df.sample(5)

,accession_id,virus_name,virus_code,collection_date,submission_date,year,month,day,quarter,time_period,region,clade,variant,aa_substitutions
109,EPI_ISL_592143,hCoV-19/Australia/VIC12997/2020,VIC-00012997,2020-08-01,2020-10-22,2020,8,1,3,3,Victoria,GR,NaN,"(Spike_S477N,N_R203K,NSP15_S261P,N_G204R,NSP2_..."
454,EPI_ISL_7987421,hCoV-19/Australia/NSW-ICPMR-17597/2021,NSW-ICPMR-00017597,2021-12-12,2021-12-23,2021,12,12,4,8,New South Wales,GK,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,Spike_T95I,NS3_A59V,N_D63G,N_R203M,NS..."
93,EPI_ISL_663953,hCoV-19/Australia/VIC17624/2020,VIC-00017624,2020-07-25,2020-11-30,2020,7,25,3,3,Victoria,GR,NaN,"(N_R203K,N_G204R,NSP2_I120F,NSP12_P323L,Spike_..."
263,EPI_ISL_3833749,hCoV-19/Australia/NSW-ICPMR-5136/2021,NSW-ICPMR-00005136,2021-08-20,2021-09-01,2021,8,20,3,7,New South Wales,GK,VOC Delta GK (B.1.617.2+AY.*) first detected i...,"(N_G215C,Spike_T95I,N_D63G,N_R203M,NSP12_G671S..."
26,EPI_ISL_456630,hCoV-19/Australia/VIC1731/2020,VIC-00001731,2020-04-29,2020-06-02,2020,4,29,2,2,Victoria,O,NaN,"(NSP2_V198I,NSP2_P91S,NS8_S24L,NSP6_A287V,NSP3..."


### AA-Substitution Vectorization

The following cells build binary vectors for each known substitution.

#### Extract the Amino Acid Substitutions ("AA_Substitutions" column)

Each `aa_substitution` identifier string takes the form:
    
    {gene}_{reference_aa}{position}{substitute_aa}

This is split into the constituent parts to produce a multi-leveled column, ordered as:

1. Gene
2. Position
3. Reference AA
4. Substituted AA

Note the inversion of the position and reference aa.

Then, each row describes the presence (or lack thereof) of the substitution within the sample.

For example, `spike_D614G` would be split into a hierarchical column:

1. Spike
2. 614
3. D
4. G

and would contain a `0` in `EPI_ISL_402125` (the reference sequence), \
but a `1` for delta variant samples.

In [22]:
%%time
if aa_subs_by_seq_fpath.exists():
    aa_subs_by_seq_df = pd.read_csv(
        aa_subs_by_seq_fpath,
        index_col=index_levels,
        header=header_levels
        # index_names=index_names
    ).fillna(0)
else:
    # iterate through substitutions, remove parentheses, split by comma. 
    # If substitution exists, mark with `1`
    aa_subs = {
        idx: pd.Series(
            [1 for _ in row[1:-1].split(',')], index=row[1:-1].split(',')
        ) for idx, row in au_reduced_df.set_index(index_names)['aa_substitutions'].items()
    }
    aa_subs_by_seq_df = pd.DataFrame.from_dict(aa_subs, orient='index')
    aa_subs_by_seq_df = aa_subs_by_seq_df.fillna(0).sort_index(axis=1).drop(columns='', errors='ignore')
    aa_subs_by_seq_df.index = aa_subs_by_seq_df.index.rename(index_names)
    aa_subs_by_seq_df = aa_subs_by_seq_df.astype('int32')

    # create levels
    aa_multi_cols = aa_subs_by_seq_df.columns.str.extractall(r"(?P<gene>.+?)_(?P<aa_ref>([A-Z]+?)|(ins))(?P<position>[0-9]+)(?P<aa_sub>.+)")
    aa_multi_cols = aa_multi_cols.fillna('').droplevel(1).drop([2,3], axis=1)
    aa_multi_cols = aa_multi_cols.set_index(aa_subs_by_seq_df.columns)
    aa_multi_cols = aa_multi_cols[['gene', 'position', 'aa_ref', 'aa_sub']]
    
    # Apply to new dataframe
    aa_subs_by_seq_df.columns = pd.MultiIndex.from_tuples(
        aa_multi_cols.values.tolist(), 
        names=aa_multi_cols.columns.tolist()
    )

    aa_subs_by_seq_df = aa_subs_by_seq_df.sort_index(axis=1).sort_index(axis=0)

    if '.xz' not in str(aa_subs_by_seq_fpath):
        # create a fake "sparse" table to reduce character count
        aa_subs_by_seq_df.replace(0, '').to_csv(aa_subs_by_seq_fpath)
    else:
        aa_subs_by_seq_df.to_csv(aa_subs_by_seq_fpath)


CPU times: user 42.4 ms, sys: 294 µs, total: 42.7 ms
Wall time: 40.5 ms


In [40]:
aa_subs_by_seq_df.sample(5)

gene                                                                                E  \
position                                                                           14   
aa_ref                                                                              V   
aa_sub                                                                            del   
region            collection_date virus_code         accession_id    time_period        
Victoria          2020-07-17      VIC-00009767       EPI_ISL_565703  3            0.0   
Western Australia 2022-01-21      WA-00001054        EPI_ISL_9163967 9            0.0   
New South Wales   2021-09-12      NSW-ICPMR-00007866 EPI_ISL_4552282 7            0.0   
                  2021-12-04      NSW-ICPMR-00016361 EPI_ISL_7621421 8            0.0   
                  2021-08-09      NSW-ICPMR-00003290 EPI_ISL_3426161 7            0.0   

gene                                                                                   \
position                                                                           62   
aa_ref                                                                              V   
aa_sub                                                                              F   
region            collection_date virus_code         accession_id    time_period        
Victoria          2020-07-17      VIC-00009767       EPI_ISL_565703  3            0.0   
Western Australia 2022-01-21      WA-00001054        EPI_ISL_9163967 9            0.0   
New South Wales   2021-09-12      NSW-ICPMR-00007866 EPI_ISL_4552282 7            0.0   
                  2021-12-04      NSW-ICPMR-00016361 EPI_ISL_7621421 8            0.0   
                  2021-08-09      NSW-ICPMR-00003290 EPI_ISL_3426161 7            0.0   

gene                                                                                   \
position                                                                           68   
aa_ref                                                                              S   
aa_sub                                                                              F   
region            collection_date virus_code         accession_id    time_period        
Victoria          2020-07-17      VIC-00009767       EPI_ISL_565703  3            0.0   
Western Australia 2022-01-21      WA-00001054        EPI_ISL_9163967 9            0.0   
New South Wales   2021-09-12      NSW-ICPMR-00007866 EPI_ISL_4552282 7            0.0   
                  2021-12-04      NSW-ICPMR-00016361 EPI_ISL_7621421 8            0.0   
                  2021-08-09      NSW-ICPMR-00003290 EPI_ISL_3426161 7            0.0   

gene                                                                                   \
position                                                                           74   
aa_ref                                                                              L   
aa_sub                                                                              M   
region            collection_date virus_code         accession_id    time_period        
Victoria          2020-07-17      VIC-00009767       EPI_ISL_565703  3            0.0   
Western Australia 2022-01-21      WA-00001054        EPI_ISL_9163967 9            0.0   
New South Wales   2021-09-12      NSW-ICPMR-00007866 EPI_ISL_4552282 7            0.0   
                  2021-12-04      NSW-ICPMR-00016361 EPI_ISL_7621421 8            0.0   
                  2021-08-09      NSW-ICPMR-00003290 EPI_ISL_3426161 7            0.0   

gene                                                                                   \
position                                                                            9   
aa_ref                                                                              T   
aa_sub                                                                              I   
region            collection_date virus_code         accession_id    time_period

## Summary Statistics


### Substitution Count by each Region-Period

Calculate the sum of each substitution column, partitioned by combined region-period  (e.g., New Queensland Q1)

In [41]:
aa_sub_counts_by_region_period = aa_subs_by_seq_df.groupby(level=['region', 'time_period']).sum()
aa_sub_counts_by_region_period.to_csv(aa_sub_counts_by_region_period_fname)

# fill missing time periods with "0"
regions = aa_sub_counts_by_region_period.index.get_level_values(0).unique()
max_period = aa_sub_counts_by_region_period.index.get_level_values(1).max()

aa_sub_counts_by_region_period = aa_sub_counts_by_region_period.reindex([
    (region, period) 
    for region in regions 
    for period in range(1,max_period + 1)
]).fillna(0).astype('int32')


In [42]:
aa_sub_counts_by_region_period.sample(10)

gene                                       E               M                 \
position                                  14 62 68 74  9 101 162 168 175 19   
aa_ref                                     V  V  S  L  T   R   K   I   T  Q   
aa_sub                                   del  F  F  M  I   K   N   L   M  E   
region                       time_period                                      
Queensland                   7             0  0  0  0  0   0   0   0   0  0   
Tasmania                     7             0  0  0  0  0   0   0   0   0  0   
Australian Capital Territory 6             0  0  0  0  0   0   0   0   0  0   
South Australia              8             0  0  0  0  1   0   0   0   0  1   
Western Australia            8             0  0  0  0  0   0   0   0   0  0   
Northern Territory           6             0  0  0  0  0   0   0   0   0  0   
South Australia              3             0  0  0  0  0   0   0   0   0  0   
Queensland                   3             0  0  0  0  0   0   0   0   0  0   
Australian Capital Territory 4             0  0  0  0  0   0   0   0   0  0   
Tasmania                     8             0  0  0  0  0   0   0   0   0  0   

gene                                      ... Spike                         \
position                                  ...   937 95 950 954 964 969 981   
aa_ref                                    ...     S  T   D   Q   K   N   L   
aa_sub                                    ...     L  I   N   H   E   K   F   
region                       time_period  ...                                
Queensland                   7            ...     0  0   3   0   0   0   0   
Tasmania                     7            ...     0  0   0   0   0   0   0   
Australian Capital Territory 6            ...     0  0   0   0   0   0   0   
South Australia              8            ...     1  3   2   1   0   1   1   
Western Australia            8            ...     0  0   0   0   0   0   0   
Northern Territory           6            ...     0  0   1   0   0   0   0   
South Australia              3            ...     0  0   0   0   0   0   0   
Queensland                   3            ...     0  0   0   0   0   0   0   
Australian Capital Territory 4            ...     0  0   0   0   0   0   0   
Tasmania                     8            ...     0  0   0   0   0   0   0   

gene                                                  
position                                 982 983 986  
aa_ref                                     S   R   K  
aa_sub                                     A   C   Q  
region                       time_period              
Queensland                   7             0   0   0  
Tasmania                     7             0   0   0  
Australian Capital Territory 6             0   0   0  
South Australia              8             0   0   0  
Western Australia            8             0   0   0  
Northern Territory           6             1   0   0  
South Australia              3             0   0   0  
Queensland                   3             0   0   0  
Australian Capital Territory 4             0   0   0  
Tasmania                     8             0   0   0  

[10 rows x 749 columns]

### Sequence Counts by Regional Period

Calculate the number of Australian sequences, partitioned by combined region-period

In [43]:
%%time
if seq_counts_fpath.exists():
    ## This will take at least 7 seconds to retrieve
    seq_counts_df = pd.read_csv(
        seq_counts_fpath,
        index_col=short_index_names
    )
else:
    seq_counts_df = aa_subs_by_seq_df.droplevel([1,2]).index.to_frame()
    
    seq_counts_df = seq_counts_df.reset_index(drop=True)
    seq_counts_df = seq_counts_df.groupby(short_index_names).count()
    seq_counts_df = seq_counts_df.rename(columns={'accession_id': 'counts'})

    seq_counts_df.to_csv(seq_counts_fpath)

# fill missing time periods with "0"
regions = seq_counts_df.index.get_level_values(0).unique()
max_period = seq_counts_df.index.get_level_values(1).max()

seq_counts_df = seq_counts_df.reindex([
    (region, period) 
    for region in regions 
    for period in range(1,max_period + 1)
]).fillna(0).astype('int32')

CPU times: user 9.64 ms, sys: 0 ns, total: 9.64 ms
Wall time: 6.49 ms


In [57]:
seq_counts_df.sample(10)

region                        time_period
Western Australia             8               0
Tasmania                      2               1
Queensland                    3               0
Tasmania                      8               0
Northern Territory            2               0
                              8               1
South Australia               9               5
Queensland                    6               3
Tasmania                      5               0
Australian Capital Territory  8              13
Name: counts, dtype: int32

## Test

Find incidence of Spike D614G across region-periods.

In [79]:
aa_sub_counts_by_region_period.div(seq_counts_df['counts'].values, axis=0).loc[
    :,
    aa_sub_counts_by_region_period.columns.get_level_values('position') == '614'
].T


region                       Australian Capital Territory                      \
time_period                                             1   2   3   4   5   6   
gene  position aa_ref aa_sub                                                    
Spike 614      D      G                               1.0 NaN NaN NaN NaN NaN   

region                                      New South Wales  ... Victoria  \
time_period                     7    8    9               1  ...        9   
gene  position aa_ref aa_sub                                 ...            
Spike 614      D      G       1.0  1.0  1.0            0.25  ...      1.0   

region                       Western Australia                                \
time_period                                  1    2   3   4    5   6   7   8   
gene  position aa_ref aa_sub                                                   
Spike 614      D      G                    0.5  1.0 NaN NaN  1.0 NaN NaN NaN   

region                             
time_period                     9  
gene  position aa_ref aa_sub       
Spike 614      D      G       1.0  

[1 rows x 72 columns]